Research QUestions:
-------------------------

(RQ1)  Cost comparison with more extensive benchmark dataset 

(RQ2)  Which domain specific grow helps improve the overal runtime?

(RQ3)  How does extraction multiple correction subsets affect runtime

    (a) as an initial bootstrapping method;
    (b) as a replacement for grow and its complement.

(RQ4)  What is the efficiency of a single step O(C)US and is single step sufficiently efficient for an interactive context?
    
    -> What is an “interactive context”?
    -> Is the most difficult explanation faster in one over the other?
    -> Did we have to explain the previous steps as well?

In [1]:
from pathlib import Path
from pyexplain.benchmark.file_utils import *
from pyexplain.benchmark.plot import *
import IPython
import numpy as np
pd.set_option('display.max_rows', 100)

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['font.weight']= 'bold'

In [2]:
def histogram_all_explanation_times(config_expl_times: dict, order, path=None):
    #print(config_expl_times.keys())
    max_t = 0
    for label, v in config_expl_times.items():
        max_t = max([max_t, max(v)])

    bins=[0,0.1, 0.2, 0.3, 0.4, 0.5, 1,2,3,4, 5,10,20,30, 40, 50] + list(range(100, int(max_t//100 + 2)*100,100)) 
    v = []

    #fig, axs = plt.subplots(figsize=(15, 8))
    plt.figure(figsize =(15, 8))
    colors = ['r','g','b', 'c','m','y']
    
    #for o, c in zip(order, colors):
    plt.hist([config_expl_times[o] for o in order], bins=bins, label=order, stacked=True)
 
    plt.yscale('log')
    plt.xscale('log')
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.legend()
    if path:
        plt.savefig()


In [3]:
def histogram_individual_configs_explanation_times(config_expl_times: dict, order, path=None):
    #print(config_expl_times.keys())
    max_t = 0
    for label, v in config_expl_times.items():
        max_t = max([max_t, max(v)])

    bins=[0,0.1, 0.2, 0.3, 0.4, 0.5, 1,2,3,4, 5,10,20,30, 40, 50] + list(range(100, int(max_t//100 + 2)*100,100)) 
    v = []

    #fig, axs = plt.subplots(figsize=(15, 8))

    colors = ['r','g','b', 'c','m','y']
    
    fig, axs = plt.subplots(nrows=len(order)//2,ncols=2, figsize=(20, 20))
    for id, o in enumerate(order):
        row, col = id//2, id%2
        
        axs[row, col].hist(config_expl_times[o], bins=bins, label=o, stacked=True)
        axs[row, col].set_yscale('log')
        axs[row, col].set_xscale('log')
        axs[row, col].legend(fontsize=16)
        axs[row, col].tick_params(axis = 'both', labelsize = 18)
        
 
    if path:
        plt.savefig(path)


In [4]:
# MAC
environment = 'MAC'
BASE_MAC_LINUX = {
    'MAC': '/Users/emiliogamba/Documents/01_VUB/01_Research/01_Shared_Projects',
    'LINUX': '/home/emilio/research/'
}

PATH_FIGURES_POST_PAPER = Path(BASE_MAC_LINUX[environment] + "01_holygrail/latex/journal/jair21/figures/")
EXPERIMENT_RESULTS = Path(BASE_MAC_LINUX[environment] + "06_HPC_Experiments/experiments/data/output/")
BASE_OUTPUT_PATH = BASE_MAC_LINUX[environment] + "/06_HPC_Experiments/experiments/data/output/"
REMOTE_EXPERIMENT_RESULTS = "/data/brussel/101/vsc10143/hpc_experiments2/experiments/data/output/"

In [5]:
#### Table with some information about explanation average time...
def explanation_time_quantile(t_expl, quantile):
    return round(np.quantile(t_expl, quantile), 2)

def latex_rename_corr_expl_config(expl_config):
    renaming = {
        'Iterated OUS+Incr.+Greedy CorrSubsets': 'Iter. OUS+SAT Corr. SS',
        'OCUS+Incr.+CorrSubsets': 'OCUS+Corr. SS', 
        'Iterated OUS+Incr.+Greedy CorrSubsets+Greedy Corr. Subsets Boostrap Once': 'Iter. OUS(+Bootstrap Once)+SAT Corr. SS',
        'OCUS+Incr.+CorrSubsets+Corr. Subsets Boostrap All':  'OCUS (+Bootstrap All).+Corr. SS',
        'OCUS+Incr.+MaxSAT+Greedy Corr. Subsets Boostrap All':  'OCUS (+Bootstrap All).+MaxSAT',
        'Iterated OUS+Incr.+MaxSAT+Corr. Subsets Boostrap Once': 'Iter. OUS+MaxSAT(Corr. Subsets Boostrap Once)'
    }
    return renaming[expl_config]
    

def stats_expls_times(df):
    #df["config"] = df.apply(lambda row: latex_rename_corr_expl_config(row["corr_explanation_config"]), axis=1)
    df["avg_t_expl"] = df.apply(lambda row: round(np.mean(row["t_expl"]),2), axis=1)
    df["med_t_expl"] = df.apply(lambda row: np.median(row["t_expl"]), axis=1)
    df["n_expl"] = df.apply(lambda row: len(row["t_expl"]), axis=1)
    df["min_t_expl"] =df.apply(lambda row: np.min(row["t_expl"]), axis=1)
    df["max_t_expl"] =df.apply(lambda row: np.max(row["t_expl"]), axis=1)
    #df["tot_expl"] =df.apply(lambda row: sum(row["tot_time"]), axis=1)
    df["tot_time (s)"] =df.apply(lambda row: round(row["tot_time"]), axis=1)
    df["time_to_first_expl"] =df.apply(lambda row: row["t_expl"][0], axis=1)
    df["q_25"] =df.apply(lambda row: explanation_time_quantile(row["t_expl"], quantile=0.25), axis=1)
    df["q_50"] =df.apply(lambda row: explanation_time_quantile(row["t_expl"], quantile=0.50), axis=1)
    df["q_75"] =df.apply(lambda row: explanation_time_quantile(row["t_expl"], quantile=0.75), axis=1)
    df["q_95"] =df.apply(lambda row: explanation_time_quantile(row["t_expl"], quantile=0.95), axis=1)
    df["q_98"] =df.apply(lambda row: explanation_time_quantile(row["t_expl"], quantile=0.98), axis=1)
    df["q_100"] =df.apply(lambda row: explanation_time_quantile(row["t_expl"], quantile=1), axis=1)
    return df

In [6]:
#  Optux Previsou results = 2021061714
corr_output_folders = [
    "MUS/2022030116/",
    "RQ2_LOGIC_SUDOKU_PUZZLES/2022021411/",
    "RQ2_LOGIC_SUDOKU_PUZZLES_MAXSAT_FULL/2022021708/",
    "RQ2_DEMYSTIFY_PUZZLES/2022021714",
    "RQ2_DEMYSTIFY_PUZZLES_MAXSAT_FULL/2022021708/",
    "RQ2c_DEMYSTIFY_SUDOKU_PUZZLES/2022022508/",
    "RQ2c_DEMYSTIFY_PUZZLES/2022022508/",
    "RQ2_MISSING_PUZZLES/2022030215/",
    "RQ2_MISSING_PUZZLES/2022031010/"
]

path_corr_output_dirs = [BASE_OUTPUT_PATH + o for o in corr_output_folders]
df_corr = folder_to_pandas_df_pickle(path_corr_output_dirs)
df_corr_enh = corr_enhance_df(df_corr)

renaming_expl_config = {
    "MUS": "MUS",
    "OCUS":"OCUS+Incr. HS",
    "OCUS_NOT_INCREMENTAL":"OCUS",
    "OUS_INCREMENTAL_NAIVE":"OUSb+Lit. Incr. HS",
    "OUS_INCREMENTAL_NAIVE_PARALLEL": "OUS Iter.+Lit. Incr. HS",
    "OUS_NAIVE_PARALLEL":"OUS Iter.",
    "OUS_SS": "OUSb"
}

df_corr_enh["explanation config"] = df_corr_enh.apply(lambda row: renaming_expl_config[row["params_explanation_computer"]], axis=1)
df_corr_enh["tot_lits_derived"] = df_corr_enh.apply(lambda row: sum(row["lits_derived"]), axis=1)
df_corr_enh["time_to_first_expl"] =df_corr_enh.apply(lambda row: row["params_timeout"] if len(row["time_cumul_explain"]) ==0 else row["time_cumul_explain"][0], axis=1)


/home/emilio/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/emilio/.local/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [7]:
#  Optux Previsou results = 2021061714
mus_output_folder = [
    "MUS/2022030116/",
]

path_mus_output_dirs = [BASE_OUTPUT_PATH + o for o in mus_output_folder]
df_mus = folder_to_pandas_df_pickle(path_mus_output_dirs)
df_mus_enh = corr_enhance_df(df_mus)

renaming_expl_config = {
    "MUS": "MUS",
    "OCUS":"OCUS+Incr. HS",
    "OCUS_NOT_INCREMENTAL":"OCUS",
    "OUS_INCREMENTAL_NAIVE":"OUSb+Lit. Incr. HS",
    "OUS_INCREMENTAL_NAIVE_PARALLEL": "OUS Iter.+Lit. Incr. HS",
    "OUS_NAIVE_PARALLEL":"OUS Iter.",
    "OUS_SS": "OUSb"
}

df_mus["explanation config"] = df_mus.apply(lambda row: renaming_expl_config[row["params_explanation_computer"]], axis=1)
df_mus["tot_lits_derived"] = df_mus.apply(lambda row: sum(row["lits_derived"]), axis=1)
df_mus["time_to_first_expl"] =df_mus.apply(lambda row: row["params_timeout"] if len(row["time_cumul_explain"]) ==0 else row["time_cumul_explain"][0], axis=1)


mus_isntances_no_timeout = df_mus[
    (df_mus["explanation config"] == "MUS") & 
    (df_mus["time_timedout"] == False)
]["params_instance"].to_list()
print(len(mus_isntances_no_timeout))

311


In [8]:
ocus_isntances_no_timeout =  df_corr_enh[
    (df_corr_enh["params_grow_config"] == "Greedy MCSes + Actual + Unif.") & 
    (df_corr_enh["explanation config"] == "OCUS+Incr. HS") & 
    (df_corr_enh["time_timedout"] == False)
]["params_instance"].to_list()
print(len(ocus_isntances_no_timeout))

311


Select for each O(C)US algorithm, the correction subset enuemration algorithm and group the explanations for each configuration

In [9]:
df_greedy_corr =  df_corr_enh[
    (df_corr_enh["params_grow_config"] == "Greedy MCSes + Actual + Unif.")
]

grouped_greedy_corr_configs = df_greedy_corr.groupby(
    by=["explanation config"]
).agg(
    t_expl=("time_explain", sum),
    t_avg_first_expl=("time_to_first_expl", lambda x: round(np.mean(x), 2)),
    cnt=("time_explain", "count"),
    tot_time=("tot_time_explain", np.sum)
).reset_index()


df_incr_configs_indiv_expls = stats_expls_times(
    grouped_greedy_corr_configs[
        [
            "explanation config", 
            "t_expl", 
            "t_avg_first_expl",
            "tot_time"]
    ].copy(deep=True)
)

## RQ4: What is the efficiency of a single step O(C)US and is single step sufficiently efficient for an interactive context?

In [10]:
ignored = [
    "t_expl",
    "med_t_expl",
    "n_expl",
    "min_t_expl",
    "max_t_expl",
    "corr_explanation_config",
    "tot_time"
]
all_cols = [column for column in df_incr_configs_indiv_expls.columns if column not in ignored]+["tot_time"]
df_indiv_expls_table = df_incr_configs_indiv_expls[
    [
        "explanation config", 
        #"t_avg",
        "t_avg_first_expl",
        "avg_t_expl", 
        "q_25", 
        "q_50", 
        "q_75", 
        "q_95", 
        "q_98", 
        "q_100"
    ]
]
display(df_indiv_expls_table)
print(df_indiv_expls_table.to_latex())

,explanation config,t_avg_first_expl,avg_t_expl,q_25,q_50,q_75,q_95,q_98,q_100
0,OCUS,145.59,16.91,1.03,1.86,8.10,98.32,115.31,513.07
1,OCUS+Incr. HS,146.85,11.89,0.39,0.73,28.82,36.54,39.04,383.28
2,OUS Iter.,152.97,18.25,1.72,3.33,5.61,132.65,226.99,1152.09
3,OUS Iter.+Lit. Incr. HS,182.74,12.39,0.42,0.61,29.50,48.92,52.12,1831.79
4,OUSb,499.67,13.60,2.02,3.81,6.52,41.92,173.72,3028.68
5,OUSb+Lit. Incr. HS,504.78,8.74,0.48,0.69,1.21,53.59,57.88,3428.02


\begin{tabular}{llrrrrrrrr}
\toprule
{} &       explanation config &  t\_avg\_first\_expl &  avg\_t\_expl &  q\_25 &  q\_50 &   q\_75 &    q\_95 &    q\_98 &    q\_100 \\
\midrule
0 &                     OCUS &            145.59 &       16.91 &  1.03 &  1.86 &   8.10 &   98.32 &  115.31 &   513.07 \\
1 &            OCUS+Incr. HS &            146.85 &       11.89 &  0.39 &  0.73 &  28.82 &   36.54 &   39.04 &   383.28 \\
2 &                OUS Iter. &            152.97 &       18.25 &  1.72 &  3.33 &   5.61 &  132.65 &  226.99 &  1152.09 \\
3 &  OUS Iter.+Lit. Incr. HS &            182.74 &       12.39 &  0.42 &  0.61 &  29.50 &   48.92 &   52.12 &  1831.79 \\
4 &                     OUSb &            499.67 &       13.60 &  2.02 &  3.81 &   6.52 &   41.92 &  173.72 &  3028.68 \\
5 &       OUSb+Lit. Incr. HS &            504.78 &        8.74 &  0.48 &  0.69 &   1.21 &   53.59 &   57.88 &  3428.02 \\
\bottomrule
\end{tabular}



## Repeat same computation but only on instances that did not timeout

ALL configurations are considered

In [11]:
df_greedy_corr =  df_corr_enh[
    (df_corr_enh["params_grow_config"] == "Greedy MCSes + Actual + Unif.")
]

## select instances that do not timeout for all configurations
grouped_params_instance_summed = df_greedy_corr.groupby(
    by=["params_instance"]
).sum().reset_index()

non_timedout_instances = grouped_params_instance_summed[
    grouped_params_instance_summed["time_timedout"] == 0
]["params_instance"].to_list()


print(len(non_timedout_instances))

308


In [12]:
## select instances that do not timeout for incremental configurations
df_greedy_corr_incremental =  df_corr_enh[
    (df_corr_enh["params_grow_config"] == "Greedy MCSes + Actual + Unif.") & 
    (df_corr_enh["explanation config"].isin([
        'OCUS+Incr. HS',
        'OUS Iter.+Lit. Incr. HS',
        'OUSb+Lit. Incr. HS'
    ]))
]

## select instances that do not timeout for all configurations
grouped_params_instance_summed_incremental = df_greedy_corr_incremental.groupby(
    by=["params_instance"]
).sum().reset_index()

# non_timedout_instances_incremental = set(df_corr_enh["params_instance"])
non_timedout_instances_incremental = grouped_params_instance_summed_incremental[
    grouped_params_instance_summed_incremental["time_timedout"] == 0
]["params_instance"].to_list()


df_greedy_corr_incremental_non_timeout = df_greedy_corr_incremental[
    df_greedy_corr_incremental["params_instance"].isin(non_timedout_instances_incremental)
]

df_greedy_corr_incremental_non_timeout_grouped = df_greedy_corr_incremental_non_timeout.groupby(
    by=["explanation config"]
).agg(
    t_expl=("time_explain", sum),
    t_avg_first_expl=("time_to_first_expl", lambda x: round(np.mean(x), 2)),
    cnt=("time_explain", "count"),
    tot_time=("tot_time_explain", np.sum)
).reset_index()


df_greedy_corr_incremental_non_timeout_indiv_expls = stats_expls_times(
    df_greedy_corr_incremental_non_timeout_grouped[
        [
            "explanation config", 
            "t_expl", 
            "t_avg_first_expl",
            #"t_avg", 
            "tot_time"]
    ].copy(deep=True)
)

ignored = [
    "t_expl",
    "med_t_expl",
    "n_expl",
    "min_t_expl",
    "max_t_expl",
    "corr_explanation_config",
    "tot_time"
]
all_cols = [column for column in df_greedy_corr_incremental_non_timeout_indiv_expls.columns if column not in ignored]+["tot_time"]
df_indiv_expls_table = df_greedy_corr_incremental_non_timeout_indiv_expls[
    [
        "explanation config", 
        "avg_t_expl", 
        "t_avg_first_expl",
        "q_25", 
        "q_50", 
        "q_75", 
        "q_95", 
        "q_98", 
        "q_100"
    ]
]
display(df_indiv_expls_table)
print(df_indiv_expls_table.to_latex(index=False))

,explanation config,avg_t_expl,t_avg_first_expl,q_25,q_50,q_75,q_95,q_98,q_100
0,OCUS+Incr. HS,1.87,8.24,0.34,0.46,0.76,5.26,16.84,342.03
1,OUS Iter.+Lit. Incr. HS,0.88,7.90,0.38,0.52,0.67,1.72,5.68,180.10
2,OUSb+Lit. Incr. HS,1.35,22.09,0.45,0.62,0.85,2.40,7.53,2568.67


\begin{tabular}{lrrrrrrrr}
\toprule
     explanation config &  avg\_t\_expl &  t\_avg\_first\_expl &  q\_25 &  q\_50 &  q\_75 &  q\_95 &  q\_98 &   q\_100 \\
\midrule
          OCUS+Incr. HS &        1.87 &              8.24 &  0.34 &  0.46 &  0.76 &  5.26 & 16.84 &  342.03 \\
OUS Iter.+Lit. Incr. HS &        0.88 &              7.90 &  0.38 &  0.52 &  0.67 &  1.72 &  5.68 &  180.10 \\
     OUSb+Lit. Incr. HS &        1.35 &             22.09 &  0.45 &  0.62 &  0.85 &  2.40 &  7.53 & 2568.67 \\
\bottomrule
\end{tabular}



In [13]:
## select instances that do not timeout for non-incremental configurations
df_greedy_corr_not_incremental =  df_corr_enh[
    (df_corr_enh["params_grow_config"] == "Greedy MCSes + Actual + Unif.") & 
    (df_corr_enh["explanation config"].isin([
        'OCUS',
         'OUS Iter.',
         'OUSb',
    ]))
]

## select instances that do not timeout for all configurations
grouped_params_instance_summed_not_incremental = df_greedy_corr_not_incremental.groupby(
    by=["params_instance"]
).sum().reset_index()

## compute same with timeout included
# non_timedout_instances_not_incremental = set(df_corr_enh["params_instance"])
non_timedout_instances_not_incremental = grouped_params_instance_summed_not_incremental[
    grouped_params_instance_summed_not_incremental["time_timedout"] == 0
]["params_instance"].to_list()


##### INSTANCES FROM INCREMENTAL THAT DO NOT TIMEOUT
df_greedy_corr_not_incremental_non_timeout = df_greedy_corr_not_incremental[
    df_greedy_corr_not_incremental["params_instance"].isin(non_timedout_instances_not_incremental)
]

df_greedy_corr_not_incremental_non_timeout_grouped = df_greedy_corr_not_incremental_non_timeout.groupby(
    by=["explanation config"]
).agg(
    t_expl=("time_explain", sum),
    t_avg_first_expl=("time_to_first_expl", lambda x: round(np.mean(x), 2)),
    #t_avg=("avg_t_explain", np.mean),
    cnt=("time_explain", "count"),
    tot_time=("tot_time_explain", np.sum)
).reset_index()


df_greedy_corr_not_incremental_non_timeout_indiv_expls = stats_expls_times(
    df_greedy_corr_not_incremental_non_timeout_grouped[
        [
            "explanation config", 
            "t_expl", 
            "t_avg_first_expl",
            #"t_avg", 
            "tot_time"]
    ].copy(deep=True)
)

ignored = [
    "t_expl",
    "med_t_expl",
    "n_expl",
    "min_t_expl",
    "max_t_expl",
    "corr_explanation_config",
    "tot_time"
]
all_cols = [column for column in df_greedy_corr_not_incremental_non_timeout_indiv_expls.columns if column not in ignored]+["tot_time"]
df_indiv_expls_table = df_greedy_corr_not_incremental_non_timeout_indiv_expls[
    [
        "explanation config", 
        #"t_avg",
        "t_avg_first_expl",
        "avg_t_expl", 
        "q_25", 
        "q_50", 
        "q_75", 
        "q_95", 
        "q_98", 
        "q_100"
    ]
]
display(df_indiv_expls_table)
print(df_indiv_expls_table.to_latex())

,explanation config,t_avg_first_expl,avg_t_expl,q_25,q_50,q_75,q_95,q_98,q_100
0,OCUS,6.87,4.04,0.93,1.56,3.34,14.06,20.78,400.73
1,OUS Iter.,3.22,4.56,1.62,3.09,5.03,12.75,18.93,388.95
2,OUSb,13.57,5.46,1.95,3.63,6.01,13.51,23.50,893.17


\begin{tabular}{llrrrrrrrr}
\toprule
{} & explanation config &  t\_avg\_first\_expl &  avg\_t\_expl &  q\_25 &  q\_50 &  q\_75 &   q\_95 &   q\_98 &   q\_100 \\
\midrule
0 &               OCUS &              6.87 &        4.04 &  0.93 &  1.56 &  3.34 &  14.06 &  20.78 &  400.73 \\
1 &          OUS Iter. &              3.22 &        4.56 &  1.62 &  3.09 &  5.03 &  12.75 &  18.93 &  388.95 \\
2 &               OUSb &             13.57 &        5.46 &  1.95 &  3.63 &  6.01 &  13.51 &  23.50 &  893.17 \\
\bottomrule
\end{tabular}



## RQ4: What is the efficiency of a single step O(C)US and is single step sufficiently efficient for an interactive context?

In [14]:
ignored = [
    "t_expl",
    "med_t_expl",
    "n_expl",
    "min_t_expl",
    "max_t_expl",
    "corr_explanation_config",
    "tot_time"
]
all_cols = [column for column in df_incr_configs_indiv_expls.columns if column not in ignored]+["tot_time"]
df_indiv_expls_table = df_incr_configs_indiv_expls[
    [
        "explanation config", 
        #"t_avg",
        "t_avg_first_expl",
        "avg_t_expl", 
        "q_25", 
        "q_50", 
        "q_75", 
        "q_95", 
        "q_98", 
        "q_100"
    ]
]
display(df_indiv_expls_table)

,explanation config,t_avg_first_expl,avg_t_expl,q_25,q_50,q_75,q_95,q_98,q_100
0,OCUS,145.59,16.91,1.03,1.86,8.10,98.32,115.31,513.07
1,OCUS+Incr. HS,146.85,11.89,0.39,0.73,28.82,36.54,39.04,383.28
2,OUS Iter.,152.97,18.25,1.72,3.33,5.61,132.65,226.99,1152.09
3,OUS Iter.+Lit. Incr. HS,182.74,12.39,0.42,0.61,29.50,48.92,52.12,1831.79
4,OUSb,499.67,13.60,2.02,3.81,6.52,41.92,173.72,3028.68
5,OUSb+Lit. Incr. HS,504.78,8.74,0.48,0.69,1.21,53.59,57.88,3428.02
